# Deep Learning  For Text and Sequences

### dl models processing text(seq of words or characters), timeseries or seq of data in general. 2 fund dl algos r recurrent neural
networks and ID convnets. 
Apps:-
    document classification and timeseries classification, such as identifying the topi or author
    timeseries comparisons such as estimating how closely related two documents or two stock tickers are
    se1 2 seq learning such as deconding an English sentence into French
    Sentiment analysis such as classifying the sentiment of tweets or movie reviews as +ve or -ve
    Timeseries forecasting, such as predicting the future weather at a certain location, given recent weather data
    
E.gs:-
    sentiment analysis on the IMDB dataset
    temperature forecasting

#### 6.1 Working with text data

#### dl 4 nlp is patten recognition applied 2 words, sentences & paragraphs in much the same way dat omputer vision is pattern
recognition applied 2 pixels. numeric tensors fed into nns as input rather than raw text. Vectorizing text is da process
of transforming text in2 numeric tensors. this can b done in multiple ways:-
    segment text into words, and transform each word into a vector
    segment text into characters and transform each character into a vector
    extract n-grams of words or characters, and transform each n-gram into a vector. N-grams r overlapping groups of multiple
    consecutive groups or characters.

Collectively, the different units in which u can break down text(words, character, or n-grams) are called tokens, and breaking text in2 such tokens is called tokenization. All text-vectorization processes consist of applying some tokenization scheme
and then associating numeric vectors with the generated tokens. These vectors, packed into sequence tokens are fed in2 deep nns.There r multiple ways 2 associate a vector with a token. In this section, I will present 2 major ones: one-hot encoding of
tokens and token-embedding(usd excl 4 words kald word embedding). 

### Understanding n-grams & bag of words

## 6.1.1 One hot-encding of words and characters

## 6.1 Word-level one-hot encoding(toy-example)

In [27]:
import numpy  as np

samples = ['The cat sat on the mat.', 'The dog ate my homework.' ] # initial data: one entry/sample(here a sample is a sentence
                                                                   # bit it cud b an entire document)
    
token_index = {}     #Builds an index of all tokens in the data
for sample in samples:         
    for word in sample.split():  # Tokenizes da sample via split method. In real life, strip punctuaton & special characters.
        if word not in token_index:
            token_index[word] = len(token_index) + 1  #assigns uniq index 2each uniq word. index 0 nt attributed 2 anything.
            
    max_length = 10    # vectorizes the samples. only consider da 1st max_length words in each sample.
    
    results = np.zeros(shape=(len(samples),
                              max_length,
                              max(token_index.values()) + 1))   # results r stored here
    
    for i, sample in enumerate(samples):
        for j, word in list(enumerate(sample.split()))[:max_length]:
            index = token_index.get(word)
            results[i, j, index] = 1
            


                      

In [28]:
token_index

{'The': 1,
 'cat': 2,
 'sat': 3,
 'on': 4,
 'the': 5,
 'mat.': 6,
 'dog': 7,
 'ate': 8,
 'my': 9,
 'homework.': 10}

## 6.2 Character-level one-hot encoding

In [29]:
import string

samples = ['The cat sat on the mat.', 'The dog ate my homework.']
characters = string.printable   # All printable ASCII charcters
token_index = dict(zip(range(1, len(characters) + 1), characters))

max_length = 50
results = np.zeros((len(samples), max_length, max(token_index.keys()) + 1))
for i, sample in enumerate(samples):
    for j, character in enumerate(sample):
        index = token_index.get(character)
        results[i, j, index] = 1

In [30]:
results

array([[[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

# Keras has buitlin utilites 4 one-hot encoding of text @ word/character levels starting 4rm raw data. should use them

# 6.3 Using Keras for word-level one-hot encoding

In [31]:
from keras.preprocessing.text import Tokenizer

samples = ['The cat sat on the mat.', 'The dog ate my homework.']

tokenizer = Tokenizer(num_words=1000)     #Creates a tokenizer, configd to take only into account the 1000 most common words
tokenizer.fit_on_texts(samples)         # Builds the word index

sequences = tokenizer.texts_to_sequences(samples)    #Turns strings into lists of integer indices

one_hot_results = tokenizer.texts_to_matrix(samples, mode='binary')  #Cud also directly get 1-hot binary reps.
                        # Vectorization modes other than one-hot encoding are supported b y this tokenizer
    
word_index = tokenizer.word_index     # How u can recover the word index that was computed

print('Found %s unique tokens.' % len(word_index))  

Found 9 unique tokens.


# One-hot hashing trick

# 6.4 Word-Level one-hot encoding with hashing trick (toy example)

In [32]:
samples = ['The cat sat on the mat.', 'The dog ate my homework' ]

dimensionality = 1000    # Stores the words as vectors of size 1000. If u've close 2 1000 words(or more), u'll see many hash 
                         # collissions, which will decrease the accuracy of this encoding method.
max_length= 10

results = np.zeros((len(samples), max_length, dimensionality))
for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        index = abs(hash(word)) % dimensionality  # Hashes the word into a random index integer btw 0 and 1000
        results[i, j, index] = 1

In [33]:
results

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

# 6.1.2 Using word embeddings

### Learning word embeddings with the embedding layer

# 6.5 Instantiating an Embedding Layer

In [9]:
from keras.layers import Embedding
embedding_layer = Embedding(1000, 64)  #The Embedding Layer takes at least 2 args: the no. of possible tokens(here, 1,000
                                       # 1 + maximum word index) and the dimensionality of the embeddings(here, 64)
    
#The embedding layer is best understodd as a dictionary that maps integer indices (which stand for specific words) 2 dense vectors.
#It takes integers as input, it luks up these integers in an internal dictionary, and it returns the associated vectors. it's
#effectively a dictionary lookup.

# word index => Embedding layer => Corresponding word vector

#the embedding layer takes as input a 2D tensor of integers ,of shape( samples, sequence_length), where each entry is a seq of
# integers. it can embed sequences of variable lengths: i.e. u cud feed in2 da Embedding Layer in the previous example batches
# with shapes (32, 10)(batch of 32 sequences of length 10) or (64, 15)(batch of 64 sequences of length 15). All seqs in a 
# batch mst have the same length, though(as they have 2b packed in2 a single tensor), so seqs shorter than others shud b
# padded with zeros, and seqs dat r longer shud be truncated.

# dis layer return a 3D floating point tensor of shape (samples, sequence_length, embedding_dimensionality). Such a 3D tensor
# can then b processed by an RNN layer or a 1D convolution layer. 

# wen u instantiate an EMbedding latyer, its weights (its internal dict of token vectors) are initally random, just as with 
# any other layer. Dring training, these word vectors are gradually adjusted via backpropagation, structuring the space in2 sth
# the downstream model can exploit. Once fully trained, the embedding space will show a lot of structure- a kind of structure
# specialized for the specific problem 4 which ur training ur model.

# let's apply his idea to the IMDB movie-review sentiment prediction task. 1st. prepare data. restrict move review 2 top 10000
# mst common words and cut off the review after only 20 words. THe network will learn 8 dimensiaol embdngs 4 each of the 10000
# words, turn the input integer sequences (2D integer tensor) in2 embedded sequences (3D float tensor), flatten the tensor 2
# 2D, and train a single Dense layer on top 4 classification.

# 6.6 Loading the IMDB data for use with an Embedding Layer

In [10]:
from keras.datasets import imdb
from keras import preprocessing

max_features = 10000   # no. of words 2 consider as features
maxlen = 20     #  cuts off the text after this no. of words(among the max_features most common words)

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words = max_features)    # loads da data as list of integerz


x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)   # turns the list of integers in2 a 2D integer tensor 
                                                                         # of shape (samples, maxlen)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

# 6.7 Using an  Embedding Layer and classifier on the IMDB data

In [11]:
from keras.models import Sequential
from keras.layers import Flatten, Dense

model = Sequential()
model.add(Embedding(10000, 8, input_length=maxlen))  #Specifies da max input length 2da Embedding layer so u can later flatten
                        # da embedded inputs. Aftr da Embedding layer, the activations have shape (samples, maxlen, 8)

model.add(Flatten())    # Flattens the 3D tensor of embeddings in2 a 2D tensor of shape(samples, maxlen * 8)

model.add(Dense(1, activation='sigmoid'))       # adds the classifier on top
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()

history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 8)             80000     
_________________________________________________________________
flatten (Flatten)            (None, 160)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 161       
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 3s 2ms/step - loss: 0.6863 - acc: 0.5657 - val_loss: 0.6218 - val_acc: 0.6982
Epoch 2/10
625/625 [==============================] - 1s 1ms/step - loss: 0.5695 - acc: 0.7444 - val_loss: 0.5229 - val_acc: 0.7312
Epoch 3/10
625/625 [==============================] - 1s 1ms/step - loss: 0.4684 - acc: 0.7834 - val_loss: 0.4996 - 

# Using Pretrained Word Embeddings

# Putting it all together: 4rm raw text to word embeddings

# Downloading the IMDB data as raw text

# 6.8 Processing the labels of the raw IMDB data

In [36]:
import os

imdb_dir = 'C:/Users/Omar/Downloads/aclImdb/aclImdb/test/'
train_dir = os.path.join(imdb_dir, 'train')
 
labels = []
texts = []

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname))
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:/Users/Omar/Downloads/aclImdb/aclImdb/test/train\\neg'